# 🍷 Interactive Wine Preference Inference Engine

**Real-Time Sensitivity Analysis & Decision Boundary Exploration**

---

## 📖 How to Use This Tool

This notebook provides an **interactive sensitivity analysis tool** for exploring wine preferences using In-Context Learning. It allows you to:

### **Core Capabilities:**

1. **Real-Time Prediction** 🎯
   - Adjust 5 wine feature sliders (Acidity, Minerality, Fruitiness, Tannin, Body)
   - See instant match score updates (0-100 scale)
   - View dynamic radar chart overlaying your input with liked/disliked profiles

2. **Sensitivity Analysis** 📊
   - Discover which features most impact preference
   - Test counterfactuals: "What if this wine had more minerality?"
   - Identify minimum acceptable thresholds

3. **Decision Boundary Exploration** 🔬
   - Examine "borderline wines" that mix preferred and non-preferred traits
   - Test edge cases: Can high fruitiness compensate for low acidity?
   - Understand interaction effects between features

### **Use Cases:**

- **Wine Shopping**: Simulate a wine before buying to predict compatibility
- **Sommelier Training**: Learn which dimensions drive preference
- **Model Validation**: Test if predictions align with intuition
- **Preference Mapping**: Visualize your "acceptable region" in flavor space

---

### **Interpretation Guide:**

| Match Score | Interpretation | Action |
|-------------|----------------|--------|
| **90-100** | Exceptional Match | Buy multiple bottles |
| **75-89** | Strong Match | Confident purchase |
| **60-74** | Moderate Match | Worth trying |
| **40-59** | Uncertain | Risky - may not enjoy |
| **0-39** | Poor Match | Avoid |

---

In [ ]:
# Import libraries
import sys
import warnings
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from vinologic.predictor import VinoPredictor, PalateMatch
from vinologic.schema import WineFeatures

warnings.filterwarnings('ignore')

# Custom CSS for dark theme
display(HTML("""
<style>
    .widget-label { color: #e0e0e0 !important; font-weight: bold; }
    .widget-slider { background: #1e1e1e !important; }
    .jupyter-widgets { background: #0d1117 !important; }
</style>
"""))

print("✓ Libraries loaded")
print("✓ Dark mode theme applied")

## 1. Initialize the Prediction Engine

Load the VinoPredictor with your historical wine preferences.

In [ ]:
# Initialize predictor
print("Initializing VinoPredictor with In-Context Learning...")
predictor = VinoPredictor()

# Load historical data for visualization
df = predictor.df
feature_cols = ['acidity', 'minerality', 'fruitiness', 'tannin', 'body']

# Calculate average profiles
liked_avg = df[df['liked'] == True][feature_cols].mean()
disliked_avg = df[df['liked'] == False][feature_cols].mean() if False in df['liked'].values else None

print(f"\n✓ Engine ready with {len(predictor.liked_examples)} liked + {len(predictor.disliked_examples)} disliked examples")
print("\nPreference Profile (Average Liked Wines):")
print(liked_avg)

## 2. Interactive Prediction Interface

Use the sliders below to simulate a wine's flavor profile and see real-time predictions.

In [ ]:
# Premium dark theme configuration
DARK_THEME = {
    'paper_bgcolor': '#0d1117',
    'plot_bgcolor': '#161b22',
    'font_color': '#e6edf3',
    'grid_color': 'rgba(56, 139, 253, 0.15)',
    'liked_color': 'rgba(56, 139, 253, 0.8)',  # Electric blue
    'disliked_color': 'rgba(248, 113, 113, 0.8)',  # Coral red
    'current_color': 'rgba(163, 230, 53, 0.9)',  # Lime green
}

# Create interactive sliders
acidity_slider = widgets.IntSlider(value=7, min=1, max=10, description='Acidity:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
minerality_slider = widgets.IntSlider(value=6, min=1, max=10, description='Minerality:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
fruitiness_slider = widgets.IntSlider(value=6, min=1, max=10, description='Fruitiness:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
tannin_slider = widgets.IntSlider(value=5, min=1, max=10, description='Tannin:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
body_slider = widgets.IntSlider(value=6, min=1, max=10, description='Body:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))

# Output widgets
output_chart = widgets.Output()
output_score = widgets.Output()

def create_radar_chart(current_features):
    """Create premium dark-theme radar chart."""
    fig = go.Figure()
    
    categories = ['Acidity', 'Minerality', 'Fruitiness', 'Tannin', 'Body']
    
    # Liked wines profile
    liked_vals = liked_avg.tolist() + [liked_avg.iloc[0]]
    fig.add_trace(go.Scatterpolar(
        r=liked_vals,
        theta=categories + [categories[0]],
        fill='toself',
        fillcolor='rgba(56, 139, 253, 0.2)',
        line=dict(color=DARK_THEME['liked_color'], width=2),
        name='✓ Liked Wines',
        marker=dict(size=6)
    ))
    
    # Disliked wines profile (if available)
    if disliked_avg is not None:
        disliked_vals = disliked_avg.tolist() + [disliked_avg.iloc[0]]
        fig.add_trace(go.Scatterpolar(
            r=disliked_vals,
            theta=categories + [categories[0]],
            fill='toself',
            fillcolor='rgba(248, 113, 113, 0.15)',
            line=dict(color=DARK_THEME['disliked_color'], width=2),
            name='✗ Disliked Wines',
            marker=dict(size=6)
        ))
    
    # Current wine (from sliders)
    current_vals = current_features + [current_features[0]]
    fig.add_trace(go.Scatterpolar(
        r=current_vals,
        theta=categories + [categories[0]],
        fill='toself',
        fillcolor='rgba(163, 230, 53, 0.25)',
        line=dict(color=DARK_THEME['current_color'], width=3),
        name='⭐ Current Wine',
        marker=dict(size=8, symbol='star')
    ))
    
    # Apply dark theme
    fig.update_layout(
        polar=dict(
            bgcolor=DARK_THEME['plot_bgcolor'],
            radialaxis=dict(
                visible=True,
                range=[0, 10],
                showticklabels=True,
                tickfont=dict(size=11, color=DARK_THEME['font_color']),
                gridcolor=DARK_THEME['grid_color'],
            ),
            angularaxis=dict(
                tickfont=dict(size=13, color=DARK_THEME['font_color'], family='Arial Black'),
            )
        ),
        showlegend=True,
        title=dict(
            text='<b>Real-Time Flavor Profile Analysis</b>',
            font=dict(size=18, color=DARK_THEME['font_color']),
            x=0.5,
            xanchor='center'
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.2,
            xanchor='center',
            x=0.5,
            font=dict(size=12, color=DARK_THEME['font_color']),
            bgcolor='rgba(22, 27, 34, 0.8)'
        ),
        paper_bgcolor=DARK_THEME['paper_bgcolor'],
        height=550,
        width=750,
        margin=dict(t=80, b=80)
    )
    
    return fig

def update_prediction(change):
    """Update prediction when sliders change."""
    # Get current values
    features_dict = {
        'acidity': acidity_slider.value,
        'minerality': minerality_slider.value,
        'fruitiness': fruitiness_slider.value,
        'tannin': tannin_slider.value,
        'body': body_slider.value
    }
    
    current_features = [features_dict[col] for col in feature_cols]
    
    # Create WineFeatures object
    wine_features = WineFeatures(**features_dict, reasoning="Interactive simulation")
    
    # Build ICL prompt
    context_prompt = predictor._build_context_prompt(wine_features)
    
    # Get prediction
    try:
        completion = predictor.client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": "You are an expert sommelier analyzing wine palate compatibility."},
                {"role": "user", "content": context_prompt}
            ],
            response_format=PalateMatch,
            temperature=0.5
        )
        match = completion.choices[0].message.parsed
        match_score = match.match_score
        analysis = match.qualitative_analysis
        recommendation = match.recommendation
    except Exception as e:
        match_score = 50
        analysis = f"Unable to analyze (API error: {str(e)[:100]})"
        recommendation = "N/A"
    
    # Update chart
    with output_chart:
        output_chart.clear_output(wait=True)
        fig = create_radar_chart(current_features)
        fig.show()
    
    # Update score display
    with output_score:
        output_score.clear_output(wait=True)
        
        # Color code based on score
        if match_score >= 75:
            color = '#10b981'  # Green
            emoji = '✅'
            verdict = 'STRONG MATCH'
        elif match_score >= 60:
            color = '#f59e0b'  # Amber
            emoji = '⚠️'
            verdict = 'MODERATE MATCH'
        else:
            color = '#ef4444'  # Red
            emoji = '❌'
            verdict = 'POOR MATCH'
        
        # Display score card
        display(HTML(f"""
        <div style="background: linear-gradient(135deg, #161b22 0%, #0d1117 100%);
                    border: 2px solid {color};
                    border-radius: 15px;
                    padding: 25px;
                    margin: 20px 0;
                    box-shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.5);">
            <div style="text-align: center;">
                <h2 style="color: {color}; margin: 0; font-size: 2.5em;">{emoji} {verdict}</h2>
                <h1 style="color: {color}; margin: 10px 0; font-size: 4em; font-weight: bold;">{match_score:.0f}/100</h1>
                <p style="color: #e6edf3; font-size: 1.1em; margin-top: 20px; line-height: 1.6;">{analysis}</p>
                <div style="margin-top: 20px; padding: 15px; background: rgba(56, 139, 253, 0.1); border-radius: 8px;">
                    <p style="color: #58a6ff; font-weight: bold; margin: 0;">💡 {recommendation}</p>
                </div>
            </div>
        </div>
        """))

# Attach listeners
acidity_slider.observe(update_prediction, 'value')
minerality_slider.observe(update_prediction, 'value')
fruitiness_slider.observe(update_prediction, 'value')
tannin_slider.observe(update_prediction, 'value')
body_slider.observe(update_prediction, 'value')

# Initial update
update_prediction(None)

# Display UI
print("🎛️ Interactive Prediction Dashboard")
print("Adjust sliders to see real-time predictions\n")

display(widgets.VBox([
    acidity_slider,
    minerality_slider,
    fruitiness_slider,
    tannin_slider,
    body_slider
]))

display(output_score)
display(output_chart)

## 3. Decision Boundary Analysis

Explore borderline wines that mix preferred and non-preferred characteristics.

In [ ]:
# Decision Boundary Test Cases
borderline_wines = [
    {
        'name': 'High Acid + High Fruit',
        'features': {'acidity': 9, 'minerality': 5, 'fruitiness': 9, 'tannin': 4, 'body': 6},
        'hypothesis': 'Can high fruit compensate for moderate minerality if acidity is high?'
    },
    {
        'name': 'High Mineral + High Body',
        'features': {'acidity': 6, 'minerality': 9, 'fruitiness': 5, 'tannin': 6, 'body': 9},
        'hypothesis': 'Does high body hurt the match even with great minerality?'
    },
    {
        'name': 'Balanced Profile',
        'features': {'acidity': 7, 'minerality': 7, 'fruitiness': 7, 'tannin': 6, 'body': 7},
        'hypothesis': 'What happens with a perfectly balanced wine?'
    },
    {
        'name': 'Low Acid + High Mineral',
        'features': {'acidity': 4, 'minerality': 9, 'fruitiness': 5, 'tannin': 5, 'body': 7},
        'hypothesis': 'Can ultra-high minerality save a low-acid wine?'
    }
]

print("🔬 Testing Decision Boundaries...\n")
results = []

for wine in borderline_wines:
    features = WineFeatures(**wine['features'], reasoning="Boundary test")
    context_prompt = predictor._build_context_prompt(features)
    
    try:
        completion = predictor.client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": "You are an expert sommelier analyzing wine palate compatibility."},
                {"role": "user", "content": context_prompt}
            ],
            response_format=PalateMatch,
            temperature=0.5
        )
        match = completion.choices[0].message.parsed
        score = match.match_score
    except Exception as e:
        print(f"⚠️ API error for {wine['name']}: {str(e)[:50]}")
        score = 50.0
    
    results.append({
        'Wine': wine['name'],
        'Acid': wine['features']['acidity'],
        'Mineral': wine['features']['minerality'],
        'Fruit': wine['features']['fruitiness'],
        'Body': wine['features']['body'],
        'Score': score,
        'Hypothesis': wine['hypothesis']
    })

# Create results table
results_df = pd.DataFrame(results)
print(results_df[['Wine', 'Acid', 'Mineral', 'Fruit', 'Body', 'Score']].to_string(index=False))
print("\n" + "="*80)

# Visualize decision boundaries
fig = go.Figure()

for idx, result in enumerate(results):
    wine_name = result['Wine']
    score = result['Score']
    features_list = [result['Acid'], result['Mineral'], result['Fruit'], result['Body'], 5]  # Tannin placeholder
    
    # Color based on score
    if score >= 75:
        color = 'rgba(56, 139, 253, 0.7)'
    elif score >= 60:
        color = 'rgba(251, 191, 36, 0.7)'
    else:
        color = 'rgba(248, 113, 113, 0.7)'
    
    fig.add_trace(go.Scatterpolar(
        r=features_list + [features_list[0]],
        theta=['Acidity', 'Minerality', 'Fruitiness', 'Body', 'Tannin', 'Acidity'],
        name=f"{wine_name} ({score:.0f})",
        line=dict(color=color, width=2),
        marker=dict(size=6),
        fill='toself',
        fillcolor=color.replace('0.7', '0.15')
    ))

fig.update_layout(
    polar=dict(
        bgcolor=DARK_THEME['plot_bgcolor'],
        radialaxis=dict(
            visible=True,
            range=[0, 10],
            tickfont=dict(size=10, color=DARK_THEME['font_color']),
            gridcolor=DARK_THEME['grid_color']
        ),
        angularaxis=dict(
            tickfont=dict(size=12, color=DARK_THEME['font_color'])
        )
    ),
    title=dict(
        text='<b>Decision Boundary Exploration</b>',
        font=dict(size=18, color=DARK_THEME['font_color']),
        x=0.5
    ),
    legend=dict(
        font=dict(size=11, color=DARK_THEME['font_color']),
        bgcolor='rgba(22, 27, 34, 0.8)'
    ),
    paper_bgcolor=DARK_THEME['paper_bgcolor'],
    height=600,
    showlegend=True
)

fig.show()

print("\n📊 Key Insights:")
print(f"  • Highest score: {results_df['Score'].max():.0f} ({results_df.loc[results_df['Score'].idxmax(), 'Wine']})")
print(f"  • Lowest score: {results_df['Score'].min():.0f} ({results_df.loc[results_df['Score'].idxmin(), 'Wine']})")
print(f"  • Score range: {results_df['Score'].max() - results_df['Score'].min():.0f} points")

# Show hypotheses
print("\n💡 Hypothesis Validation:")
for _, row in results_df.iterrows():
    print(f"\n{row['Wine']} (Score: {row['Score']:.0f}/100)")
    print(f"  Q: {row['Hypothesis']}")
    if row['Score'] >= 75:
        print(f"  A: ✅ Strong positive signal")
    elif row['Score'] >= 60:
        print(f"  A: ⚠️ Moderate compatibility")
    else:
        print(f"  A: ❌ Likely not preferred")